# Imports

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

# Read data

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_excel("../data/B2P Dataset_2020.10.xlsx")
df.head(1)

Bridge Name Bridge Opportunity: Project Code  \
0     Bukinga                          1009317   

  Bridge Opportunity: Needs Assessment Bridge Opportunity: Level 1 Government  \
0         Rwanda Needs Assessment 2018                      Southern Province   

  Bridge Opportunity: Level 2 Government Bridge Opportunity: Stage  \
0                              Nyaruguru                 Cancelled   

   Bridge Opportunity: GPS (Latitude)  Bridge Opportunity: GPS (Longitude)  \
0                           -2.760833                            29.488056   

  Bridge Opportunity: Bridge Type  Bridge Opportunity: Span (m)  \
0                Suspended Bridge                           NaN   

   Bridge Opportunity: Individuals Directly Served  \
0                                              NaN   

            Bridge Opportunity: Comments                  Form: Form Name  \
0  Steep slopes to deal with for anchors  Project Assessment - 2018.10.10   

                Form: Created By  Proposed Bridge Location (GPS) (Latitude)  \
0  aimablengirabakunzi taroworks                                  -2.760833   

   Proposed Bridge Location (GPS) (Longitude) Current crossing method  \
0                                   29.488056                     NaN   

  Nearest all-weather crossing point  Days per year river is flooded  \
0                                NaN                             NaN   

  Flood duration during rainy season Market access blocked by river  \
0                                NaN                            NaN   

  Education access blocked by river Health access blocked by river  \
0                               NaN                            NaN   

  Other access blocked by river Primary occupations Primary crops grown  \
0                           NaN                 NaN                 NaN   

   River crossing deaths in last 3 years  \
0                                    NaN   

   River crossing injuries in last 3 years Incident descriptions  \
0                                      NaN                   NaN   

  Notes on social information Cell service quality 4WD Accessibility  \
0                         NaN                  NaN               NaN   

  Name of nearest city Name of nearest paved or sealed road  \
0                 Huye                                  NaN   

  Bridge classification Flag for Rejection  \
0                   NaN                Yes   

                                    Rejection Reason Bridge Type  \
0  There is a planned vehicular road with a vehic...         NaN   

   Estimated span (m)  Height differential between banks  \
0                 NaN                                NaN   

  Bridge Opportunity: General Project Photos Bridge Opportunity: CaseSafeID  \
0                                        NaN             006f100000asFErAAM   

  Senior Engineering Review Conducted  
0                                 Yes

# Parse

In [3]:
def parse_comments(cell):
    if type(cell) == str:
        cell = cell.split(",")
        data = {}
        data["people_served"] = cell[0].split(" people directly served")[0].split()[-1]
        data["elevation"] = cell[1].split(" Elevation:")[-1]
        data["cell"] = cell[2].split("Cell:")[-1]
        data["people_crossing_daily"] = cell[3].split("on a normal day")[-1][1:-7]
        data["injuries"] = cell[3].split("Death")[1].split()[0][1:]
        data["deaths"] = cell[3].split("/ ")[-1].split()[0]
        data["near_city_centers"] = cell[4].split("centers-")[1]
        data['how_to_cross_river'] = cell[5].split(" / ")[0].split(' Crossing River ')[-1]
        data['len_impossible_dangerous_to_cross'] = cell[6].split(" Impossible/Dangerous to cross the river-")[-1].split(" /")[0]
        data['time_to_nearest_safe_passage'] = cell[7].split("river crossing-")[1].split()[0]
        data['hours_to_reach_hospital'] = cell[8].split("Hospital-")[-1].split()[0]
        data['hours_to_reach_medical_center'] = cell[9].split("Health Center-")[-1]
        data['hours_to_reach_market'] = cell[10].split("market-")[-1]
        data["hours_to_reach_elementary"] = cell[11].split("School-")[-1]
        data["hours_to_reach_midlle_school"] = cell[12].split("School-")[-1]
        data["hours_to_reach_church"] = cell[13].split("Church-")[-1]
        data["land_type_within_50m"] = cell[14].split("of river ")[1].split()[0]
        data['soil_type'] = cell[15].split("-")[1].split("/")[0]
        data['sand_availible'] = cell[16].split("-")[1].split(" / ")[0]
        data['gravel_availible'] = cell[17].split("-")[1]
        data['stone_availible'] = cell[19].split("-")[1].split()[0]
        data['timber_availible'] = cell[20].split("-")[1].split(" / ")[0]
        data['stone_provided_by'] = cell[21].split("by-")[1].split("/")[0]
        data['sand_provided_by'] = cell[22].split("by-")[1].split("/")[0]
        data['gravel_provided_by'] = cell[23].split("by-")[1].split("/")[0]
        data['timber_provided_by'] = cell[24].split("by-")[1].split("/")[0]
        data['cement_provided_by'] = cell[25].split("by-")[1].split("/")[0]
        data['steel_provided_by'] = cell[26].split("by-")[1].split("/")[0]
        data['land_owner'] = cell[27].split("ownership-")[1].split()[0]
        data['ownership_permission'] = cell[28].split("permission-")[1].split()[0]
        data['general_comments'] = " ".join(cell[29:]).split("General Comments-")[1]
    else:
        return np.nan
    return data

In [4]:
lens = [len(i.split(",")) if type(i) == str else 0 for i in df['Bridge Opportunity: Comments']]
count = [(i, lens.count(i)) for i in set(lens)]
count

[(0, 1078),
 (1, 100),
 (33, 5),
 (35, 3),
 (34, 4),
 (32, 24),
 (36, 1),
 (3, 14),
 (8, 1),
 (2, 18),
 (4, 5),
 (37, 1),
 (5, 2),
 (28, 3),
 (29, 2),
 (30, 23),
 (31, 188)]

In [5]:
new_dfs = None
indices_to_check = []
for i in range(len(df['Bridge Opportunity: Comments'])):
    j = df['Bridge Opportunity: Comments'][i]
    if type(j) == str:
        try:
            data = parse_comments(j)
            if new_dfs is not None:
                temp = pd.DataFrame({key: [value] for key, value in data.items()})
                new_dfs = pd.concat([new_dfs,temp])
            else:
                new_dfs = pd.DataFrame({key: [value] for key, value in data.items()})
        except:
            indices_to_check.append(i)
new_dfs.head()

people_served elevation             cell people_crossing_daily injuries  \
0    6000-10000     1630m  Karambi-Gasumba                  >600     many   
0     3000-6000     2101m          Rugogwe               300-600     Many   
0      500-1000     1522m           Gasave                50-150     Many   
0      500-1000     1557m      Nyabinyenga                50-150      -No   
0     3000-6000     2153m   Remera-Ruyenzi               150-300     Many   

  deaths       near_city_centers         how_to_cross_river  \
0      2     -Karambi -Nyamagabe   now-Simple timber bridge   
0     No          -Kizimyamuriro   now-Simple timber bridge   
0     No  -Rwesero -Giti -Butare   now-Simple timber bridge   
0     No     -Cyabakamyi -Cyarwa  now-Walking through river   
0    One        -Gisanze -Remera   now-Simple timber bridge   

  len_impossible_dangerous_to_cross time_to_nearest_safe_passage  \
0                        6-9 months                           >3   
0                        3-6 months                          2-3   
0                        3-6 months                           >3   
0                        3-6 months                          2-3   
0                        6-9 months                          2-3   

  hours_to_reach_hospital hours_to_reach_medical_center hours_to_reach_market  \
0                   0.5-1                     0.5 hours             0.5 hours   
0                      >6                   0.5-1 hours           0.5-1 hours   
0                     2-6                                           1-2 hours   
0                      >6                     1-2 hours             1-2 hours   
0                      >6                     2-6 hours           0.5-1 hours   

  hours_to_reach_elementary hours_to_reach_midlle_school  \
0                 0.5 hours                    0.5 hours   
0               0.5-1 hours                  0.5-1 hours   
0               0.5-1 hours                    1-2 hours   
0               0.5-1 hours                    1-2 hours   
0                 0.5 hours                    0.5 hours   

  hours_to_reach_church land_type_within_50m  soil_type sand_availible  \
0             0.5 hours            bank-flat       Silt      Available   
0           0.5-1 hours           bank-hilly  hard rock      Available   
0           0.5-1 hours            bank-flat       Silt      Available   
0           0.5-1 hours            bank-flat       Silt      Available   
0             0.5 hours           bank-hilly      sandy      Available   

        gravel_availible stone_availible timber_availible stone_provided_by  \
0  Not available on site       Available        Available            Sector   
0  Not available on site       Available        Available            Sector   
0  Not available on site       Available        Available            Sector   
0  Not available on site       Available        Available            Sector   
0  Not available on site       Available        Available            Sector   

  sand_provided_by gravel_provided_by timber_provided_by  \
0           Sector           District           District   
0           Sector           District           District   
0           Sector           District           District   
0           Sector           District           District   
0           Sector           District           District   

                 cement_provided_by                 steel_provided_by  \
0  Can not be provided at the time   Can not be provided at the time    
0  Can not be provided at the time   Can not be provided at the time    
0  Can not be provided at the time   Can not be provided at the time    
0  Can not be provided at the time   Can not be provided at the time    
0  Can not be provided at the time   Can not be provided at the time    

   land_owner ownership_permission  \
0     Private                  Yes   
0  Government                  Yes   
0     Private                  Yes   
0     Private                  

In [6]:
new_dfs.shape

(203, 31)

In [7]:
pandas_profiling.ProfileReport(new_dfs)

In [8]:
# parse_comments("""6000-10000 people directly served, Elevation:1363m, Cell:Gatare-Nyamabuye, Bridge connects Nyarugenge-Gasabo, Injuries/Death-No person injured while trying to cross the river/ No person died while trying to cross the river Cross river on a normal day-300-600 people, Nearby city centers--Nyabugogo -Yanza, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-3-6 months / Hagati y?amezi atatu n?atandatu,Travel to nearest safe bridge/river crossing-1-2 hours / Hagati y?isaha 1 n?amasaha 2,Hours walking to reach the Hospital-0.5 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Available / birahaboneka,Stone-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Timber-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 8m.-The proposed bridge span is approximately 32m.-The level difference between two banks is 0.04m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is not sufficient-There is confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is showing a sign of erosion and it is located between flat areas.-The soil from the site is silt for both sides.-The bridge site was formerly called Yanza.""")
# """Not Duplicate 3000-6000 people directly served, Elevation:1523m, Cell:Gasave-Rwesero, Bridge would connecting Nyamagabe-Ruhango, Injuries/Death-Many peoples injured while trying to cross the river/ More than 4peoples died while trying to cross the river Cross river on a normal day-50-150 people, Nearby city centers--Masizi -Buhanda, Crossing River now-Swim / BarogaBoat / Bakoresha ubwato, Impossible/Dangerous to cross the river-9-12 months / Hagati y?amezi icyenda na cuminabiri,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital-2-6 hours,Hours walking to reach the Health Center-1-2 hours,Hours of walking to reach the market-2-6 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 20m.-The proposed bridge span is approximately 100m.-The level difference between two banks is 0.1m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is sufficient-There is no confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is showing a sign of erosion and it is located between two flat areas.-The soil from the site is silt for both sides. <<On 25th March, 2019 _In the beginning of scaling, this site was rejected by the assessor thinking that it will be a very long span bridge out of B2P standard but having the plan to survey Nyakabuye - Nkomane over a joining river"Nkomane", this Nyakabuye - Mwogo site over Mwogo river was surveyed and found to be technically feasible. >>>""".split(",")
# """1000-3000 people directly served, Elevation:1360m, Cell:Rurenge-Nkoma, Injuries/Death-Many peoples injured while trying to cross the river/ More than 3 peoples died while trying to cross the river. Cross river on a normal day-150-300 people, Nearby city centers--Ibare -Rurenge, Crossing River now-Walking through river / Banyura mumazi, Impossible/Dangerous to cross the river-9-12 months / Hagati y?amezi icyenda na cuminabiri,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital-2-6 hours,Hours walking to reach the Health Center-0.5-1 hours,Hours of walking to reach the market-1-2 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Not abvailable / Ntibihaboneka.,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 2m.-The proposed bridge span is approximately 80m.-The level difference between two banks is 0.01m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is not sufficient-There is no confluence area near the place.-The river bed at the site is unstable, there is a possibility of erosion.-The river bank of the site is showing any sign of erosion and it is located between flat areas.-The soil from the site is silt for both sides.""".split(",")
# """6000-10000 people directly served, Elevation:1363m, Cell:Gatare-Nyamabuye, Bridge connects Nyarugenge-Gasabo, Injuries/Death-No person injured while trying to cross the river/ No person died while trying to cross the river Cross river on a normal day-300-600 people, Nearby city centers--Nyabugogo -Yanza, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-3-6 months / Hagati y?amezi atatu n?atandatu,Travel to nearest safe bridge/river crossing-1-2 hours / Hagati y?isaha 1 n?amasaha 2,Hours walking to reach the Hospital-0.5 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Available / birahaboneka,Stone-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Timber-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 8m.-The proposed bridge span is approximately 32m.-The level difference between two banks is 0.04m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is not sufficient-There is confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is showing a sign of erosion and it is located between flat areas.-The soil from the site is silt for both sides.-The bridge site was formerly called Yanza.""".split(",")

In [9]:
# indices_to_check

In [10]:
fix_manual = []
for j in range(len(df["Bridge Opportunity: Comments"])):
    i  = df['Bridge Opportunity: Comments'][j]
    try:
        if type(i) == str:
            parse_comments(i)
    except:
        fix_manual.append(j)

In [11]:
# fix_manual

In [12]:
df['Bridge Opportunity: Comments'][fix_manual[0]]

'Steep slopes to deal with for anchors'

# Write the ones that worked to file

In [14]:
new_dfs.to_csv("../data/2013_2014.csv")

In [27]:
# fix_manual

In [24]:
for i in fix_manual:
#     for j in df['Bridge Opportunity: Comments'][i]:
    if find in df['Bridge Opportunity: Comments'][i]:
        print(i)
#         print(j)
#     _ = input("\n\nEnter anything when you want to move onto the next one")
#     print("------------------\n")

365


In [23]:
find = """The proposed bridge location is near the traditional crossing point at a distance of approximately 4m.-The proposed bridge span is approximately 45m."""

In [25]:
fix_manual.index(365)

69

In [29]:
df['Bridge Opportunity: Comments'][365]

'1000-3000 people directly served, Elevation:1393m, Cell:Nyabitekeri, Injuries/Death-No person injured while trying to cross the river/ No person died while trying to cross the river. Cross river on a normal day-150-300 people, Nearby city centers--Nyabitekeri, Crossing River now-Walking through river / Banyura mumaziSimple timber bridge / hari uduti, Impossible/Dangerous to cross the river-6-9 months / Hagati y?amezi atandatu n?icyenda,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital-2-6 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-hard rock/ Urutare,Sand-Available / birahaboneka,Gravel-Not abvailable / Ntibihaboneka.,Stone-Available / birahaboneka,Tim

In [ ]:
need to put index 70+